<a href="https://colab.research.google.com/github/barradasgabriel/praxis/blob/main/analise_valor_unidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

def analisar_dados_arquivo_texto(caminho_arquivo_txt, caminho_arquivo_saida):
    """
    Lê um arquivo de texto (formatado como CSV com ';' como separador),
    calcula o valor médio e a soma total de 'Vlr_Frete', a soma total de 'Qtd_Transp',
    e os valores unitários de frete, agrupados por 'Mes_Base', 'CO_Fabrica', 'CO_Cliente' e 'Incoterm'.
    Adiciona colunas com valor médio unitário e os maiores e menores valores unitários de frete,
    e o total do valor de frete.
    Salva o resultado em um novo arquivo CSV.

    Args:
        caminho_arquivo_txt (str): O caminho para o arquivo de texto de entrada.
        caminho_arquivo_saida (str): O caminho para salvar o arquivo CSV de resultados.

    Returns:
        pandas.DataFrame: Um DataFrame com os resultados agregados ou None se ocorrer um erro.
    """
    try:
        print(f"Tentando ler o arquivo de: {caminho_arquivo_txt}")
        # Lê o arquivo de texto (tratado como CSV) especificando o delimitador como ';'
        df = pd.read_csv(caminho_arquivo_txt, delimiter=';')
        print("Arquivo lido com sucesso.")

        # Verificar se as colunas esperadas existem, incluindo 'Incoterm'
        colunas_necessarias = ['Mes_Base', 'CO_Fabrica', 'CO_Cliente', 'Incoterm', 'Vlr_Frete', 'Qtd_Transp']
        print(f"Colunas encontradas no arquivo: {df.columns.tolist()}")
        for coluna in colunas_necessarias:
            if coluna not in df.columns:
                print(f"Erro: A coluna '{coluna}' não foi encontrada no arquivo.")
                return None

        print("Todas as colunas necessárias foram encontradas.")

        # Converter as colunas relevantes para os tipos numéricos corretos.
        if df['Vlr_Frete'].dtype == 'object':
            print("Convertendo 'Vlr_Frete' para numérico (tratando vírgula como decimal)...")
            df['Vlr_Frete'] = df['Vlr_Frete'].astype(str).str.replace('.', '', regex=False)
            df['Vlr_Frete'] = df['Vlr_Frete'].str.replace(',', '.', regex=False)
        df['Vlr_Frete'] = pd.to_numeric(df['Vlr_Frete'], errors='coerce')

        print("Convertendo 'Qtd_Transp' para numérico...")
        df['Qtd_Transp'] = pd.to_numeric(df['Qtd_Transp'], errors='coerce')

        print("Convertendo 'Mes_Base' para numérico...")
        df['Mes_Base'] = pd.to_numeric(df['Mes_Base'], errors='coerce')

        print("Convertendo 'CO_Fabrica' para numérico...")
        df['CO_Fabrica'] = pd.to_numeric(df['CO_Fabrica'], errors='coerce')

        print("Convertendo 'CO_Cliente' para numérico...")
        df['CO_Cliente'] = pd.to_numeric(df['CO_Cliente'], errors='coerce')
        print("Conversões de tipo concluídas.")

        # Remove linhas onde a conversão para numérico falhou
        colunas_para_dropna = ['Mes_Base', 'CO_Fabrica', 'CO_Cliente', 'Incoterm', 'Vlr_Frete', 'Qtd_Transp']
        print(f"Linhas antes de remover NaNs: {len(df)}")
        df.dropna(subset=colunas_para_dropna, inplace=True)
        print(f"Linhas após remover NaNs: {len(df)}")

        if df.empty:
            print("Erro: O DataFrame ficou vazio após remover linhas com valores inválidos. Verifique os dados nas colunas.")
            return None

        # Calcular o valor unitário do frete antes da agregação
        df['Vlr_Frete_Unitario'] = df.apply(lambda row: row['Vlr_Frete'] / row['Qtd_Transp'] if row['Qtd_Transp'] != 0 else 0, axis=1)

        # Agrupa os dados por 'Mes_Base', 'CO_Fabrica', 'CO_Cliente' e 'Incoterm'
        print("Agrupando dados e calculando as agregações...")
        dados_agregados = df.groupby(['Mes_Base', 'CO_Fabrica', 'CO_Cliente', 'Incoterm']).agg(
            Media_Vlr_Frete=('Vlr_Frete', 'mean'),
            Soma_Vlr_Frete=('Vlr_Frete', 'sum'), # Adicionando a soma do valor de frete
            Soma_Qtd_Transp=('Qtd_Transp', 'sum'),
            maior_valorF_unitario=('Vlr_Frete_Unitario', 'max'),
            menor_valorF_unitario=('Vlr_Frete_Unitario', 'min')
        ).reset_index()
        print("Agregação concluída.")

        # Calcular o valor médio por unidade
        dados_agregados['valor_Medio_unidade'] = dados_agregados.apply(
            lambda row: row['Media_Vlr_Frete'] / row['Soma_Qtd_Transp'] if row['Soma_Qtd_Transp'] != 0 else 0,
            axis=1
        )

        # Arredondar a Media_Vlr_Frete e valor_Medio_unidade
        dados_agregados['Media_Vlr_Frete'] = dados_agregados['Media_Vlr_Frete'].round(2)
        dados_agregados['valor_Medio_unidade'] = dados_agregados['valor_Medio_unidade'].round(2)
        dados_agregados['maior_valorF_unitario'] = dados_agregados['maior_valorF_unitario'].round(2)
        dados_agregados['menor_valorF_unitario'] = dados_agregados['menor_valorF_unitario'].round(2)
        dados_agregados['Soma_Vlr_Frete'] = dados_agregados['Soma_Vlr_Frete'].round(2) # Arredondando a soma do valor de frete


        print("\nResultados da Análise (incluindo Incoterm e novas colunas):")
        print(dados_agregados.to_string())

        # Salvar os resultados em um novo arquivo CSV
        try:
            if not caminho_arquivo_saida.lower().endswith('.csv'):
                caminho_arquivo_saida_csv = caminho_arquivo_saida + '.csv'
            else:
                caminho_arquivo_saida_csv = caminho_arquivo_saida

            dados_agregados.to_csv(caminho_arquivo_saida_csv, index=False, sep=';', decimal=',')
            print(f"\nResultados também foram salvos em '{caminho_arquivo_saida_csv}' no Google Colab.")
        except Exception as e:
            print(f"\nErro ao salvar o arquivo de resultados: {e}")

        return dados_agregados

    except FileNotFoundError:
        print(f"Erro CRÍTICO: O arquivo '{caminho_arquivo_txt}' não foi encontrado. Certifique-se de que ele foi carregado para o Colab e o caminho está correto.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Erro: O arquivo '{caminho_arquivo_txt}' está vazio ou não contém dados após o cabeçalho.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado durante o processamento do arquivo: {e}")
        import traceback
        traceback.print_exc()
        return None

# --- INÍCIO DA EXECUÇÃO DO SCRIPT NO COLAB ---

# Caminho do arquivo de entrada no Google Colab
# LEMBRE-SE: Você precisa fazer o upload do arquivo 'Rotas_editado.txt' para o Colab.
caminho_arquivo_entrada_colab = "/content/Rotas_editado.txt"

# Caminho e nome do arquivo de resultado no Google Colab
# O resultado será salvo como /content/Resultado.csv
caminho_arquivo_saida_colab = "/content/Resultado"

print("Iniciando a análise de dados (incluindo Incoterm e novas colunas)...")
resultados_analise_colab = analisar_dados_arquivo_texto(caminho_arquivo_entrada_colab, caminho_arquivo_saida_colab)

if resultados_analise_colab is not None:
    print("\nAnálise concluída com sucesso!")
else:
    print("\nA análise falhou. Verifique as mensagens de erro acima.")

# --- FIM DA EXECUÇÃO DO SCRIPT NO COLAB ---

Iniciando a análise de dados (incluindo Incoterm e novas colunas)...
Tentando ler o arquivo de: /content/Rotas_editado.txt
Arquivo lido com sucesso.
Colunas encontradas no arquivo: ['Dt_Pedido', 'Dt_Emissao', 'Dt_Entrega', 'Mes_Base', 'Ano_Exec', 'CO_Fabrica', 'CO_Cliente', 'Incoterm', 'Veiculo', 'Qtd/pallets', 'Qtd_Transp', 'Moeda', 'Vlr_Frete']
Todas as colunas necessárias foram encontradas.
Convertendo 'Vlr_Frete' para numérico (tratando vírgula como decimal)...
Convertendo 'Qtd_Transp' para numérico...
Convertendo 'Mes_Base' para numérico...
Convertendo 'CO_Fabrica' para numérico...
Convertendo 'CO_Cliente' para numérico...
Conversões de tipo concluídas.
Linhas antes de remover NaNs: 39516
Linhas após remover NaNs: 39516
Agrupando dados e calculando as agregações...
Agregação concluída.

Resultados da Análise (incluindo Incoterm e novas colunas):
      Mes_Base  CO_Fabrica  CO_Cliente Incoterm  Media_Vlr_Frete  Soma_Vlr_Frete  Soma_Qtd_Transp  maior_valorF_unitario  menor_valorF_un